In [1]:
import pandas as pd
import numpy as np

data = pd.read_csv('Results/RequiredDataSet.csv',header=None, names=['booking_date','channel','property_id','room_name','room_id','check_in','check_out','room_rate','booking_status'])
# data.head()


In [2]:
Results = pd.read_excel('Results/DataSetWithStayDate.xlsx')
# Results.head()

In [3]:
del Results['Old_Room_rate']
# Results.head()

In [4]:
scrapyData = pd.read_csv('Results/eventData2015.csv',header=None, names=['Day','Stay_date','Event','HolidayIn'])
scrapeResult = scrapyData.dropna()
# scrapeResult.head()

In [5]:
pd.options.mode.chained_assignment = None

In [6]:
scrapeResult['Stay_date'] =   '2015-' + scrapeResult['Stay_date'] 
# scrapeResult.head()

In [7]:
scrapeResult['Stay_date'] = pd.to_datetime(scrapeResult.Stay_date)
# scrapeResult.head()

In [8]:
del scrapeResult['HolidayIn']
del scrapeResult['Day']
# scrapeResult.head()

In [9]:
newResults = Results.filter(['Property_id','Stay_date','Booking_status','Room_rate'], axis=1)
# newResults.head()

In [10]:
newResults = newResults[(newResults.Booking_status == 'COMPLETED') | (newResults.Booking_status == 'CONFIRMED')]
# newResults.head()

In [11]:
newResults = newResults[(newResults.Stay_date >= '2015-01-01') & (newResults.Stay_date <= '2015-12-31')]
# newResults.head()

In [12]:
newResults['Stay_date'] = pd.to_datetime(newResults.Stay_date)
# newResults.head()

In [13]:
res = pd.merge(left=newResults, left_on='Stay_date',right=scrapeResult, right_on='Stay_date', how='left')

#res = pd.merge(newResults, scrapeResult, on='Stay_date', how='left')

In [14]:
res = res.sort_values(by = 'Stay_date', ascending=True)
# res1 = res.drop_duplicates()
res1 = res.reset_index()
del res1['index']
res1 = res1.fillna('-')
# res1.head()

In [ ]:
# events = res1[(res1.Event != '-') ]
# events.head()

In [15]:
events1 = res1.filter(['Property_id','Stay_date','Event','Room_rate'], axis=1)

In [16]:
events1['totalRevenue'] = events1.groupby(['Property_id','Stay_date'])['Room_rate'].transform('sum')
# events

In [17]:
events1['room_count'] = events1.groupby(['Property_id','Stay_date'])['Room_rate'].transform('count')
# events.head()

In [18]:
events1['ADR'] = events1.apply(lambda events1: (events1['totalRevenue'] / events1['room_count'] ) , axis=1)
events1.head()

,Property_id,Stay_date,Event,Room_rate,totalRevenue,room_count,ADR
0,BB,2015-01-01,New Years Day,39500.00,5.386517e+05,26,20717.371538
1,MB,2015-01-01,New Years Day,27000.00,1.177753e+06,62,18996.012336
2,KR,2015-01-01,New Years Day,45446.66,1.702727e+05,12,14189.387500
3,KR,2015-01-01,New Years Day,25976.80,1.702727e+05,12,14189.387500
4,MB,2015-01-01,New Years Day,17740.00,1.177753e+06,62,18996.012336


In [1]:
# test = events1.sort(['Room_rate'], ascending=[False])

In [19]:
del events1['Room_rate']
# del events1['Booking_status']
events1 = events1.drop_duplicates()
# events

In [20]:
events1.head()

,Property_id,Stay_date,Event,totalRevenue,room_count,ADR
0,BB,2015-01-01,New Years Day,5.386517e+05,26,20717.371538
1,MB,2015-01-01,New Years Day,1.177753e+06,62,18996.012336
2,KR,2015-01-01,New Years Day,1.702727e+05,12,14189.387500
11,MM,2015-01-01,New Years Day,9.835750e+04,9,10928.611111
12,CL,2015-01-01,New Years Day,8.416297e+05,49,17176.115510


In [35]:
events1.Property_id.unique()

array(['BB', 'MB', 'KR', 'MM', 'CL', 'CH', 'SS', 'EB', 'SV', 'KK', 'VM',
       'SC', 'BG', 'MP', 'CP', 'VV', 'GF', 'AM', 'AA'], dtype=object)

In [36]:
events1_ID = events1[events1.Property_id == input('Enter a property_id: ')]

Enter a property_id: BB


In [32]:
events1_ID_ne = events1_ID[events1_ID.Event == '-']
events1_ID_we = events1_ID[events1_ID.Event != '-']

In [33]:
import bokeh.plotting as plt
from bokeh.plotting import figure, output_file, show, ColumnDataSource
from bokeh.models import HoverTool
from bokeh.models.formatters import DatetimeTickFormatter
from bokeh.models import HoverTool,WheelZoomTool,ResetTool
# from bokeh.io import output_notebook

source = ColumnDataSource(
                                data=dict(
                                            x = events1_ID['Stay_date'],
                                            y = events1_ID['room_count'],
                                            date = [x.strftime("%Y-%m-%d") for x in events1_ID['Stay_date']],
                                            room_count = events1_ID['room_count'],
                                            effect = events1_ID['Event']
                                          )
                             )

source_ne = ColumnDataSource(
                                data=dict(
                                            x = events1_ID_ne['Stay_date'],
                                            y = events1_ID_ne['room_count'],
                                            date = [x.strftime("%Y-%m-%d") for x in events1_ID_ne['Stay_date']],
                                            room_count = events1_ID_ne['room_count'],
                                            effect = events1_ID_ne['Event']
                                          )
                             )

source_we = ColumnDataSource(
                                data=dict(
                                            x = events1_ID_we['Stay_date'],
                                            y = events1_ID_we['room_count'],
                                            date = [x.strftime("%Y-%m-%d") for x in events1_ID_we['Stay_date']],
                                            room_count = events1_ID_we['room_count'],
                                            effect = events1_ID_we['Event']
                                          )
                             )

hover = HoverTool(
                    tooltips=[
                                ("Date", "@date"),
                                ("Room Counts", "@room_count"),
#                                 ("Total revenue", "@y{1.11}"),
                                ("Event", "@effect"),
                             ]
                 )

#p = plt.figure(x_axis_type = "datetime", x_axis_label = "Date", y_axis_label = "Total Revenue", width=1400, height=700, tools=[hover], title="Total Reveneue And Weather Effects For the Property "+property_id+" For The Year "+str(year)+" "+month)
p = plt.figure(x_axis_type = "datetime", x_axis_label = "Date", y_axis_label = "Occupancy", width=900, height=700, tools=[hover,WheelZoomTool(),ResetTool()], toolbar_location="right", title="Stay Date vs Occupancy")
p.left[0].formatter.use_scientific = False
p.line('x', 'y', line_width=2, source=source, line_color = "green",legend = "Occupancy" )   
p.circle('x', 'y',line_width=2, source=source_we, line_color = "blue", legend = "With_Event" )
p.circle('x', 'y',line_width=2, source=source_ne, line_color = "red", legend = "Non_Event" )

p.ygrid.minor_grid_line_color = 'navy'
p.ygrid.minor_grid_line_alpha = 0.2
p.legend.border_line_width = 3
p.legend.border_line_color = "navy"
p.legend.border_line_alpha = 0.5

show(p)

In [24]:
import bokeh.plotting as plt
from bokeh.plotting import figure, output_file, show, ColumnDataSource
from bokeh.models import HoverTool
from bokeh.models.formatters import DatetimeTickFormatter
from bokeh.models import HoverTool,WheelZoomTool,ResetTool
# from bokeh.io import output_notebook

source = ColumnDataSource(
                                data=dict(
                                            x = events1_ID['Stay_date'],
                                            y = events1_ID['totalRevenue'],
                                            date = [x.strftime("%Y-%m-%d") for x in events1_ID['Stay_date']],
                                            room_count = events1_ID['room_count'],
                                            effect = events1_ID['Event']
                                          )
                             )

source_ne = ColumnDataSource(
                                data=dict(
                                            x = events1_ID_ne['Stay_date'],
                                            y = events1_ID_ne['totalRevenue'],
                                            date = [x.strftime("%Y-%m-%d") for x in events1_ID_ne['Stay_date']],
                                            room_count = events1_ID_ne['room_count'],
                                            effect = events1_ID_ne['Event']
                                          )
                             )

source_we = ColumnDataSource(
                                data=dict(
                                            x = events1_ID_we['Stay_date'],
                                            y = events1_ID_we['totalRevenue'],
                                            date = [x.strftime("%Y-%m-%d") for x in events1_ID_we['Stay_date']],
                                            room_count = events1_ID_we['room_count'],
                                            effect = events1_ID_we['Event']
                                          )
                             )

hover = HoverTool(
                    tooltips=[
                                ("Date", "@date"),
                                ("Room Counts", "@room_count"),
                                ("Total revenue", "@y{1.11}"),
                                ("Event", "@effect"),
                             ]
                 )

#p = plt.figure(x_axis_type = "datetime", x_axis_label = "Date", y_axis_label = "Total Revenue", width=1400, height=700, tools=[hover], title="Total Reveneue And Weather Effects For the Property "+property_id+" For The Year "+str(year)+" "+month)
p = plt.figure(x_axis_type = "datetime", x_axis_label = "Date", y_axis_label = "Total Revenue", width=900, height=700, tools=[hover,WheelZoomTool(),ResetTool()], toolbar_location="right", title="Stay Date vs Total Reveneue")
p.left[0].formatter.use_scientific = False
p.line('x', 'y', line_width=2, source=source, line_color = "green",legend = "Total Revenue" )   
p.circle('x', 'y',line_width=2, source=source_we, line_color = "blue", legend = "With_Event" )
p.circle('x', 'y',line_width=2, source=source_ne, line_color = "red", legend = "Non_Event" )

p.ygrid.minor_grid_line_color = 'navy'
p.ygrid.minor_grid_line_alpha = 0.2
p.legend.border_line_width = 3
p.legend.border_line_color = "navy"
p.legend.border_line_alpha = 0.5

show(p)

In [39]:
rev_avg_ne = events1_ID_ne["totalRevenue"].mean()
rev_avg_we = events1_ID_we["totalRevenue"].mean()


roomcount_ne = events1_ID_ne["room_count"].sum()
roomcount_we = events1_ID_we["room_count"].sum()


adr_ne = events1_ID_ne["ADR"].mean()
adr_we = events1_ID_we["ADR"].mean()

tn_ne = events1_ID_ne["room_count"].count()
tn_we = events1_ID_we["room_count"].count()

In [42]:
print("Total non events : "+str(tn_ne))
print("Total number of events  : "+str(tn_we))
print("\n")

print("Occupancy without event : "+str(roomcount_ne))
print("Occupancy with event  : "+str(roomcount_we))
print("\n")

print("ADR without event : "+str(adr_ne))
print("ADR with event  : "+str(adr_we))
print("\n")

print("Average total revenue without event : "+str(rev_avg_ne))
print("Average total revenue with event  : "+str(rev_avg_we))



Total non events : 278
Total number of events  : 87


Occupancy without event : 4269
Occupancy with event  : 1440


ADR without event : 13055.219727841142
ADR with event  : 14720.752238545681


Average total revenue without event : 216867.82176258994
Average total revenue with event  : 252972.22908045977


In [ ]:
# import numpy as np
# import pandas as pd

# Import Bokeh modules for interactive plotting
import bokeh.charts
import bokeh.charts.utils
import bokeh.io
import bokeh.models
import bokeh.palettes
import bokeh.plotting

# Display graphics in this notebook
bokeh.io.output_notebook()